# Import libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
GAUSSIAN = False
pair_no = 8
BASE_PATH = "drive/MyDrive/"
#RESULT_PATH = "optimization_result/"+str(pair_no)+"_for_all/"
RESULT_PATH = "optimization_result/RandomForSix/"
FULL_PATH = BASE_PATH + RESULT_PATH


In [3]:
!pip install SciencePlots

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torch

import os
import random
import pandas
import numpy
from scipy.optimize import curve_fit
from torch.nn import ELU
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import imageio
# seed = 7777
# random.seed(seed) 
# torch.manual_seed(seed);

In [5]:
# m = torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([.15])).sample(sample_shape = (100,10)).squeeze()
# n  = torch.distributions.uniform.Uniform(-1, +1).sample((100,10))

# print(m.shape,n.shape)

a = '10'
a.zfill(5)


'00010'

# Load Saved MP (Magnetic Parameter) Model

In [6]:
# 12 input -> 42 output
class MpModel(torch.nn.Module):
    def __init__(self):
        super(MpModel, self).__init__()

        self.linear1 = torch.nn.Linear(12, 100, bias=True)
        self.linear2 = torch.nn.Linear(100, 100, bias=True)
        self.linear3 = torch.nn.Linear(100, 42, bias=True)

    def forward(self, x):
        x = torch.atan(self.linear1(x))
        x = torch.atan(self.linear2(x))
        x = torch.atan(self.linear2(x))
        x = self.linear3(x)
        return x


mp_model = MpModel().to("cuda")
mp_model.load_state_dict(torch.load(BASE_PATH + "dwpt_v5/saved_model_state.pt"))

for param in mp_model.parameters():
    param.requires_grad = False

RuntimeError: ignored

# Create GP (Geometry Paramter) Generator

In [ ]:
elu = ELU(0)


In [ ]:
class GpModel(torch.nn.Module):
    def __init__(self, input_length: int):
        super(GpModel, self).__init__()

        self.dense_layer1 = torch.nn.Linear(int(input_length), 128)
        self.dense_layer2 = torch.nn.Linear(128, 256)
        self.dense_layer3 = torch.nn.Linear(256, 512)
        self.dense_layer4 = torch.nn.Linear(512, 256)
        self.dense_layer5 = torch.nn.Linear(256, 128)
        self.dense_layer6 = torch.nn.Linear(128, int(input_length))
        
        self.batch_norm1 = torch.nn.LazyBatchNorm1d()
        self.batch_norm2 = torch.nn.LazyBatchNorm1d()
        self.batch_norm3 = torch.nn.LazyBatchNorm1d()
        self.batch_norm4 = torch.nn.LazyBatchNorm1d()
        self.batch_norm5 = torch.nn.LazyBatchNorm1d()
        self.activation = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.batch_norm1(torch.atan(self.dense_layer1(x)))
        x = self.batch_norm2(torch.atan(self.dense_layer2(x)))
        x = self.batch_norm3(torch.atan(self.dense_layer3(x)))
        x = self.batch_norm4(torch.atan(self.dense_layer4(x)))
        x = self.batch_norm5(torch.atan(self.dense_layer5(x)))
        x = self.activation(self.dense_layer6(x))
        return x


gp_model = GpModel(10).to("cuda")
optimizer = torch.optim.Adam(gp_model.parameters(), lr=1e-4)

# System Constraints

In [ ]:
# x-direction of the primary winding (mm)
lpx_min, lpx_max = (50.0, 650.0)

# y-direction of the primary winding (mm)
lpy_min, lpy_max = (50.0, 2050.0)

# width of the primary winding (mm)
wp_min, wp_max = (25.0, 325.0)

# length of the edge of the primary core (mm)
a_min, a_max = (0.0, 200.0)

# pitch of the adjacent cores (mm)
p_min, p_max = (0.0, 200.0)

# length of the secondary winding (mm)
ls_min, ls_max = (50.0, 450.0)

# width of the secondary winding (mm)
ws_min, ws_max = (25.0, 225.0)

# input RMS current (A)
ip_min, ip_max = (50.0, 200.0)

# turn number of the primary side
np_min, np_max = (4.0, 10.0)

# turn number of the secondary side
ns_min, ns_max = (4.0, 10.0)

# switching frequency (kHz)
f = 85 * (10 ** 3)

# output power at the center (kW)
p_out = 50 * (10 ** 3)

# input DC voltage (V)
v_dc = 400

# output DC voltage (V)
v_bat = 400

# length of the edge of the secondary core
b = 50

# ???
w = 2 * numpy.pi * f  # [rad/s]

# ???
q_coil = 400

# Helper functions

In [ ]:
# Creates a tensor of shape [num, *start.shape] whose values are evenly spaced from start to end, inclusive.
# Replicates but the multi-dimensional bahaviour of numpy.linspace in PyTorch.

@torch.jit.script
def linspace(start: torch.Tensor, stop: torch.Tensor, num: int):
    # create a tensor of 'num' steps from 0 to 1
    steps = torch.arange(num, dtype=torch.float32, device=start.device) / (num - 1)

    # reshape the 'steps' tensor to [-1, *([1]*start.ndim)] to allow for broadcastings
    # - using 'steps.reshape([-1, *([1]*start.ndim)])' would be nice here but torchscript
    #   "cannot statically infer the expected size of a list in this contex", hence the code below
    for i in range(start.ndim):
        steps = steps.unsqueeze(-1)

    # the output starts at 'start' and increments until 'stop' in each dimension
    out = start[None] + steps * (stop - start)[None]

    return out

In [ ]:
# Generate a pytorch tensor full of random floats in the range [-1, +1] with the given shape
def generate_noise(shape, gaussian=False):
    if gaussian:
      return torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([.15])).sample(sample_shape = shape).squeeze().cuda()
    return torch.distributions.uniform.Uniform(-1, +1).sample(shape).cuda()

In [ ]:
def stack_parameters(*params):
    return torch.stack(params).transpose(0, 1)


# Scale an array to be between our specified [min, max] contraints
def scale(arr):
    scaler_min = torch.tensor(
        [a_min, lpx_min, lpy_min, ls_min, p_min, wp_min, ws_min, ip_min, np_min, ns_min],
        device="cuda",
    )
    scaler_max = torch.tensor(
        [a_max, lpx_max, lpy_max, ls_max, p_max, wp_max, ws_max, ip_max, np_max, ns_max],
        device="cuda",
    )
    return arr * (scaler_max - scaler_min) + scaler_min


def extract(arr):
    a, lpx, lpy, ls, p, wp, ws, ip, np, ns = [
        numpy.squeeze(x) for x in numpy.hsplit(arr, 10)
    ]

    ys_min = (
        5 * wp +
        4 * a +
        3 * lpy +
        2 * p
    )

    ys_max = (lpy + wp)
    ys_max = ys_min + (ys_min - ys_max) / 2

    ys_min /= 2
    ys_max /= 2

    ys = linspace(ys_min, ys_max, num=5)

    np = torch.round(np)
    ns = torch.round(ns)

    # take all 15 tensors of shape (X) and convert to (15, X)
    return a, lpx, lpy, ls, p, wp, ws, ip, np, ns, ys


def scale_and_extract(arr):
    scaled_array = scale(arr)
    return extract(scaled_array)

In [ ]:
# Enclose in a function so we don't leak variables
def test_models():
    noise = generate_noise(shape=(256, 10))
    gp = gp_model(noise)
    a, lpx, lpy, ls, p, wp, ws, ip, np, ns, ys = scale_and_extract(gp)

    gp_parameters = stack_parameters(a, lpx, lpy, ls, p, wp, ws, *ys)
    extra_parameters = stack_parameters(ip, np, ns)

    mp = mp_model(gp_parameters)

    print(f"""
    GP Parameters Shape:    {gp_parameters.shape}
    Extra Parameters Shape: {extra_parameters.shape}
    MP Model Output Shape:  {mp.shape}
    """)
test_models()

In [ ]:
"""
Method to take two equally-sized lists and return just the elements which lie 
on the Pareto frontier, sorted into order.
Default behaviour is to find the maximum for both X and Y, but the option is
available to specify maxX = False or maxY = False to find the minimum for either
or both of the parameters.
"""


def pareto_frontier(x, y, max_x=False):
    if max_x:
        return _pareto_frontier(y, x)
    else:
        return _pareto_frontier(x, y)


def _pareto_frontier(x, y):
    # Combine inputs and sort them with smallest X values coming first
    sorted_xy = sorted(zip(x, y))
    p_front_x = torch.zeros(len(x), device="cuda")
    p_front_y = torch.zeros(len(y), device="cuda")
    
    # Loop through the sorted list
    #   Look for lower values of Y…
    #   and add them to the Pareto frontier
    min_y = sorted_xy[0][1]
    for index in range(len(sorted_xy)):
        x, y = sorted_xy[index]
        if y < min_y:
            min_y = y
            p_front_x[index] = x
            p_front_y[index] = y
            
    p_front_x = p_front_x[p_front_x.nonzero()]
    p_front_y = p_front_y[p_front_y.nonzero()]

    return p_front_x, p_front_y

In [ ]:
def y_util(x, a, b, c):
    return (1 / (a * (x**b)))**c

In [ ]:
def distance_from_origin(*args):
  distance = 0
  for objective in args:
    distance += torch.pow(objective,2)
  
  return distance

# Load DWPT data

In [ ]:
def load_dwpt():
    folder_name = BASE_PATH + "dwpt_v5"
    file_names = [
        "DWPT_v5_N10",
        "DWPT_v5_N100",
        "DWPT_v5_N200",
        "DWPT_v5_N300",
        "DWPT_v5_N400",
    ]

    df = pandas.DataFrame()

    for file_name in file_names:
        new_df = pandas.read_csv(f"{folder_name}/{file_name}_after.csv", index_col=0)
        df = pandas.concat([df, new_df])

    return df

df = load_dwpt()
df_gp = df.loc[:, :"ys4[mm]"]
df_mp = df.loc[:, "k0mm_ys0":]

print(df_gp.shape)
print(df_mp.shape)

In [ ]:
pairs = []
file1 = open(BASE_PATH + 'dwpt_v5/pairs.txt', 'r')
lines = file1.readlines()
for line in lines:
  pairs.append([int(i) for i in line.strip().split(',')])
pair = pairs[pair_no]
print(pair)

In [ ]:
magic_number = 10 ** (-9)

# Define Loss Functions

In [ ]:
def kdiff_loss(k_parameters):
    (
        k_0_0, k_0_1, k_0_2, k_0_3, k_0_4, k_1_0,
    ) = k_parameters

    kdiff = abs(k_0_0 - k_1_0)
    kdiff = kdiff / torch.max(abs(k_0_0), abs(k_1_0))

    return kdiff


def calculate_Is(l_parameters, k_parameters):
  (
      k_0_0, k_0_1, k_0_2, k_0_3, k_0_4, k_1_0,
  ) = k_parameters
  (
      lp_0_0, lp_0_1, lp_0_2, lp_0_3, lp_0_4,
      ls_0_0, ls_0_1, ls_0_2, ls_0_3, ls_0_4,
      lp_1_0, ls_1_0,
  ) = l_parameters

  a6 = lp_0_0 * np ** 2
  a11 = ls_0_0 * ns ** 2 
  # Paper formula
  n1 = (torch.pi * w * a6 * ip) / (2 * numpy.sqrt(2) * v_dc)
  t1 = (torch.pi ** 2 * w * p_out * torch.sqrt(a6 * a11))
  t2 = 8 * k_0_0 * n1 * v_dc * v_bat
  n2 = t1 / t2
  Is = (4 * v_bat * n2) / (torch.pi * w * a11)

  return Is
  
def bstray_loss(k_parameters, l_parameters, b_parameters, extra_parameters):
    # Unpack parameters
    (
        k_0_0, k_0_1, k_0_2, k_0_3, k_0_4, k_1_0,
    ) = k_parameters
    (
        lp_0_0, lp_0_1, lp_0_2, lp_0_3, lp_0_4,
        ls_0_0, ls_0_1, ls_0_2, ls_0_3, ls_0_4,
        lp_1_0, ls_1_0,
    ) = l_parameters
    (
        bx_p_0_00, by_p_0_00, bz_p_0_00,
        bx_p_0_90, by_p_0_90, bz_p_0_90,
        bx_s_0_00, by_s_0_00, bz_s_0_00,
        bx_s_0_90, by_s_0_90, bz_s_0_90,

        bx_p_1_00, by_p_1_00, bz_p_1_00,
        bx_p_1_90, by_p_1_90, bz_p_1_90,
        bx_s_1_00, by_s_1_00, bz_s_1_00,
        bx_s_1_90, by_s_1_90, bz_s_1_90,
    ) = b_parameters
    (
        ip, np, ns
    ) = extra_parameters

    Is = calculate_Is(l_parameters,k_parameters)

    bx_100 = (
        (bx_p_1_00 * ip * np + bx_s_1_00 * Is * ns) ** 2 +
        (bx_p_1_90 * ip * np + bx_s_1_90 * Is * ns) ** 2
    ) ** 0.5
    by_100 = (
        (by_p_1_00 * ip * np + by_s_1_00 * Is * ns) ** 2 +
        (by_p_1_90 * ip * np + by_s_1_90 * Is * ns) ** 2
    ) ** 0.5
    bz_100 = (
        (bz_p_1_00 * ip * np + bz_s_1_00 * Is * ns) ** 2 +
        (bz_p_1_90 * ip * np + bz_s_1_90 * Is * ns) ** 2
    ) ** 0.5
    b_100 = (bx_100**2 + by_100**2 + bz_100**2) ** 0.5

    bstray = b_100

    return bstray

In [ ]:
import math
f = 85*10**3 #[Hz]
w =2*math.pi*f #[rad/s]
Pout= 50000 #W
Vdc = 400 #800 # V
Vbat = 400 #V
QCoil = 400
b = 50

In [ ]:
def coil_loss_and_power_average(k_parameters, l_parameters, b_parameters, extra_parameters):
  (
      k_0_0, k_0_1, k_0_2, k_0_3, k_0_4, k_1_0,
  ) = k_parameters
  (
      lp_0_0, lp_0_1, lp_0_2, lp_0_3, lp_0_4,
      ls_0_0, ls_0_1, ls_0_2, ls_0_3, ls_0_4,
      lp_1_0, ls_1_0,
  ) = l_parameters
  (
      bx_p_0_00, by_p_0_00, bz_p_0_00,
      bx_p_0_90, by_p_0_90, bz_p_0_90,
      bx_s_0_00, by_s_0_00, bz_s_0_00,
      bx_s_0_90, by_s_0_90, bz_s_0_90,

      bx_p_1_00, by_p_1_00, bz_p_1_00,
      bx_p_1_90, by_p_1_90, bz_p_1_90,
      bx_s_1_00, by_s_1_00, bz_s_1_00,
      bx_s_1_90, by_s_1_90, bz_s_1_90,
  ) = b_parameters
  (
      ip, np, ns
  ) = extra_parameters


  a6 = lp_0_0 * np**2  # LP0MM_YSO
  a7 = lp_0_1 * np**2  # LP0MM_YS1
  a8 = lp_0_2 * np**2  # LP0MM_YS2
  a9 = lp_0_3 * np**2  # LP0MM_YS3
  a10 = lp_0_4* np**2 # LP0MM_YS4

  a11 = ls_0_0 * ns**2 # LS0MM_YSO
  a12 = ls_0_1 * ns**2 # LS0MM_YS1
  a13 = ls_0_2 * ns**2 # LS0MM_YS2
  a14 = ls_0_3 * ns**2 # LS0MM_YS3
  a15 = ls_0_4 * ns**2 # LS0MM_YS4

  Is = calculate_Is(l_parameters,k_parameters)
  
  loss = (w*a6*ip**2/QCoil + w*a11*Is**2/QCoil) * 10 ** -9

  p0 = w*k_0_0*(a6*a11)**0.5*ip*Is
  p1 = w*k_0_1*(a7*a12)**0.5*ip*Is
  p2 = w*k_0_2*(a8*a13)**0.5*ip*Is
  p3 = w*k_0_3*(a9*a14)**0.5*ip*Is
  p4 = 2*w*k_0_4*(a10*a15)**0.5*ip*Is
  pave = (p0+(2*p1)+(2*p2)+(2*p3)+p4)/8 

  return loss,pave



In [ ]:
def number_of_inverters(gp_parameters):
  (
        a, lpx, lpy, ls, p, wp, ws, *ys
  ) = gp_parameters.clone().transpose(0,1)
  number_of_inverters = 1/(lpy+2*wp+2*a+p)*10**3
  return number_of_inverters

In [ ]:
  def core_losses(extra_parameters, gp_parameters):
    (
        ip, np, ns
    ) = extra_parameters

    (
        a, lpx, lpy, ls, p, wp, ws, *ys
    ) = gp_parameters.transpose(0,1)
    
    V_PriCore = ((lpy +2*wp+2*a)*(lpx+2*wp+2*a)*5)/(10**3) # cm3
    V_SecCore = (((ls+2*ws+2*b)**2)*5)/(10**3)
    V_PriWind = (2*(lpx+wp)+2*(lpy+wp))*6.6*6.6/(10**3)*np
    V_SecWind = 4*(ls+ws)*6.6*6.6/(10**3)*ns

    return V_PriCore, V_SecCore, V_PriWind, V_SecWind

In [ ]:
def loss_function_relu(params, max,min=False):

  relu = torch.nn.ReLU()
  return relu(params - max/torch.max(params))


In [ ]:
def loss_function_power_avg(params,max_val = 30000):
  max_ = torch.max(params)
  if  max_ < max_val:
    return 0
  else:
    return max_val - max_

In [ ]:
from matplotlib.patches import Rectangle

In [ ]:
noise = generate_noise(shape=(25_000, 10))
gp = gp_model(noise)

# Unpack and filter GP parameters
a, lpx, lpy, ls, p, wp, ws, ip, np, ns, ys = scale_and_extract(gp)
gp_parameters = stack_parameters(a, lpx, lpy, ls, p, wp, ws, *ys)
extra_parameters = stack_parameters(ip, np, ns).transpose(0, 1)


V_PriCore, V_SecCore, V_PriWind,V_SecWind = core_losses(extra_parameters, gp_parameters)

min_x = torch.min(gp_parameters)
max_x = torch.max(gp_parameters)
gp_parameters = (gp_parameters - min_x) / (max_x - min_x)
# Push GP parameters through MP model
mp_parameters = mp_model(gp_parameters)


# Scale MP parameters
y = torch.Tensor(df_mp.values).transpose(0, 1).to("cuda")
min_y = torch.min(y, 1).values
max_y = torch.max(y, 1).values
mp_parameters = mp_parameters * (max_y - min_y) + min_y

k_parameters = mp_parameters.transpose(0, 1)[0:6]
l_parameters = mp_parameters.transpose(0, 1)[6:18]
b_parameters = mp_parameters.transpose(0, 1)[18:]

kdiff = kdiff_loss(k_parameters)
bstray = bstray_loss(k_parameters, l_parameters, b_parameters, extra_parameters)
coilloss,pave = coil_loss_and_power_average(k_parameters, l_parameters, b_parameters, extra_parameters)
print("pave",pave,coilloss, bstray)
print(loss_function_power_avg(pave))
plt.vlines(x=40,ymin = 0, ymax=.15,color='red',linestyle='dashed')
plt.hlines(y=.15,xmin = 0, xmax=40,color='red',linestyle='dashed')
plt.vlines(x=0,ymin = 0, ymax=.15,color='red',linestyle='dashed')
plt.hlines(y=0,xmin = 0, xmax=40,color='red',linestyle='dashed')
#display plot
plt.scatter(bstray.detach().cpu(), 100 * kdiff.detach().cpu())
plt.title("Random Network: Bstray vs Kdiff")
plt.xlabel("BStray")
plt.ylabel("Coupling %")
plt.show()

plt.vlines(x=2000,ymin = 0, ymax=.15,color='red',linestyle='dashed')
plt.hlines(y=.15,xmin = 0, xmax=2000,color='red',linestyle='dashed')
plt.vlines(x=0,ymin = 0, ymax=.15,color='red',linestyle='dashed')
plt.hlines(y=0,xmin = 0, xmax=2000,color='red',linestyle='dashed')
plt.scatter(coilloss.detach().cpu(), 100 * kdiff.detach().cpu())
plt.title("Random Network: Coilloss vs Kdiff")
plt.xlabel("Coilloss")
plt.ylabel("Coupling %")
plt.show()

number_of_inverters(gp_parameters)

#Plotting

In [ ]:
fig_width = 8 #cm # Setting for Conference paper 
fig_height = 3 #cm
font_size = 7 # pt
fig_update = True
marker_size = 5
x_tick_pad = 2
y_tick_pad = 2
x_label_pad = 0.5
y_label_pad = 1

In [ ]:
import matplotlib
import seaborn as sns
plt.style.use(['science','no-latex'])
def plot_(**kwargs):
  import numpy as np
  plt.close('all')
  kdiff = kwargs["kdiff"] * 100
  pave = kwargs["pave"] /1000
  coilloss = kwargs["coilloss"]
  bstray = kwargs["bstray"]
  vpricore = kwargs["vpricore"]
  vsecwind = kwargs["vsecwind"]
  n_inv = kwargs["n_inv"]
  vseccore = kwargs["vseccore"]
  epoch = kwargs["epoch"]

  marker_size = 1
  

  font = {'family' : 'normal',
          'weight' : 'bold',
          'size'   : 12}

  matplotlib.rc('font', **font)
  
  #plt.style.use(['science','no-latex'])

  fig, (ax1, ax2, ax3,ax4) = plt.subplots(1, 4)

  #fig.subplots_adjust(hspace=10)

  fig.set_size_inches(16,4)
  #ax1.set_aspect('equal')
  #fig.tight_layout(pad=6)
  fig.tight_layout(pad = 2)

  xlim = 1 #Number of Inverter [1/m]
  ylim = 2000 #Coil loss [W] 
  ax1.set_xlabel(r"$Number\ of\ inverter\ [1/m]$", labelpad = x_label_pad)              # not shown
  ax1.set_ylabel(r'$Coil loss [W]$', labelpad = y_label_pad) # not shown
  x1 = np.arange(0, xlim + 0.1,0.1)
  y1 = 0
  y2 = ylim
  plt_ax1 = ax1.scatter(n_inv, coilloss, s=marker_size, rasterized=True)
  #fig.colorbar(mappable = plt_ax1 , ax = ax1, label=r"$B_{\rm stray}~[\rm \mu T]$")
  ax1.fill_between(x1, y1, y2 ,facecolor='r',alpha=0.3)
  ax1.plot([xlim, xlim], [0, ylim], 'r--', lw=0.5)  
  ax1.plot([0,xlim], [ylim, ylim], 'r--', lw=0.5) 
  ax1.axis([0,2, 0,5000])
  ax1.tick_params(axis='x', pad=15)
  # ax1.set_aspect('equal')



  xlim = 100
  ylim = 40
  ax2.set_xlabel(r"$B_{\rm stray}~[\rm \mu T]$", labelpad = x_label_pad)              # not shown
  ax2.set_ylabel(r'$Coupling [\%]$', labelpad = y_label_pad) # not shown

  x1 = np.arange(0, xlim + 0.1, 0.1)
  y1 = 0
  y2 = ylim

  plt_ax2 = ax2.scatter(bstray, kdiff,s=marker_size,rasterized=True)
  #fig.colorbar(mappable = plt_ax2 , ax = ax2, label=r"$V_{PriCore}$")
  ax2.fill_between(x1, y1, y2 ,facecolor='r',alpha=0.3)
  ax2.plot([xlim, xlim], [0, ylim], 'r--', lw=0.5)  
  ax2.plot([0,xlim], [ylim, ylim], 'r--', lw=0.5) 
  ax2.axis([0, 200, 0, 60])
  ax2.tick_params(axis='x', pad=15)

  # ax2.set_aspect('equal')

  xlim = 3500
  ylim = 30

  ax3.set_xlabel(r"$V_{SecCore}$", labelpad = x_label_pad)              # not shown
  ax3.set_ylabel(r'$P_{ave} [KW/M]$', labelpad = y_label_pad) # not shown

  x1 = np.arange(0, xlim + 0.1, 0.1)
  y1 = ylim
  y2 = ylim*10

  ax3.scatter(vseccore, pave,s=marker_size, rasterized=True)
  #fig.colorbar(mappable = plt_ax3 , ax = ax3, label=r"$V_{SecWind}$")
  ax3.fill_between(x1, y1, y2 ,facecolor='r',alpha=0.3)
  ax3.plot([xlim, xlim], [ylim, ylim*10], 'r--', lw=0.5)  
  ax3.plot([0,xlim], [ylim, ylim], 'r--', lw=0.5) 
  ax3.axis([0, 5000, 0, 50])
  ax3.tick_params(axis='x', pad=15)
  # ax3.set_aspect('equal')


  xlim = 750
  ylim = 6000

  ax4.set_xlabel(r"$V_{SecWind}$", labelpad = x_label_pad)              # not shown
  ax4.set_ylabel(r'$V_{PriCore} $', labelpad = y_label_pad) # not shown

  x1 = np.arange(0, xlim + 0.1, 0.1)
  y1 = 0
  y2 = ylim

  ax4.scatter(vsecwind, vpricore, s=marker_size, rasterized=True)
  #fig.colorbar(mappable = plt_ax3 , ax = ax3, label=r"$V_{SecWind}$")
  ax4.fill_between(x1, y1, y2 ,facecolor='r',alpha=0.3)
  ax4.plot([xlim, xlim], [0, ylim], 'r--', lw=0.5)  
  ax4.plot([0,xlim], [ylim, ylim], 'r--', lw=0.5) 
  ax4.axis([0, 2000, 0, 8000])
  ax4.tick_params(axis='x', pad=15)

  plt.suptitle("Epoch "+str(epoch).zfill(5), y = 1.05, c='r')
  plt.savefig(FULL_PATH + str(epoch).zfill(5)+".png")

  plt.clf()
  


In [ ]:
def accepted_solution(**kwargs):

  kdiff = kwargs["kdiff"] * 100
  pave = kwargs["pave"]/1000
  coilloss = kwargs["coilloss"]
  bstray = kwargs["bstray"]
  vpricore = kwargs["vpricore"]
  vsecwind = kwargs["vsecwind"]
  n_inv = kwargs["n_inv"]
  vseccore = kwargs["vseccore"]

  count = 0
  for i in range(len(kdiff)):
    if n_inv[i] < 1 and coilloss[i] < 2000 and bstray[i] < 50 and \
    kdiff[i] < 50 and vseccore[i] < 1500 and pave[i] > 30:# and vsecwind[i] < 750 and vpricore[i] < 6000:
      count += 1

  return count

In [ ]:
def generate_solution_plot(**kwargs):
  kdiff = kwargs["kdiff"] * 100
  pave = kwargs["pave"] /1000
  coilloss = kwargs["coilloss"]
  bstray = kwargs["bstray"]
  vpricore = kwargs["vpricore"]
  vsecwind = kwargs["vsecwind"]
  n_inv = kwargs["n_inv"]
  vseccore = kwargs["vseccore"]

  accepted_n_inv_ = [] 
  accepted_coilloss = [] 
  accepted_bstray = []
  accepted_kdiff = []
  accepted_v_sec_core = []
  accpeted_pave = []
  accepted_v_sec_wind = []
  accpeted_v_pri_core = []

  for i in range(len(kdiff)):
    if n_inv[i] < 1 and coilloss[i] < 2000 and bstray[i] < 50 and \
    kdiff[i] < 40 and vseccore[i] < 1500 and pave[i] > 30:# and vsecwind[i] < 750 and vpricore[i] < 6000:
      accepted_n_inv_.append(n_inv[i])
      accepted_coilloss.append(coilloss[i])
      accepted_bstray.append(bstray[i])
      accepted_kdiff.append(kdiff[i])
      accepted_v_sec_core.append(vseccore[i])
      accpeted_pave.append(pave[i])
      accepted_v_sec_wind.append(vsecwind[i])
      accpeted_v_pri_core.append(vpricore[i])


  import matplotlib
  import seaborn as sns
  import numpy as np

  plt.subplots_adjust(left=.1,
                      bottom=0.1,
                      right=0.9,
                      top=2,
                      wspace=2,
                      hspace=0.4)

  font = {'family' : 'normal',
          'weight' : 'bold',
          'size'   : 15}

  matplotlib.rc('font', **font)
  color_list = sns.color_palette("Paired", n_colors=len(accpeted_v_pri_core))

  plt.style.use(['science','no-latex'])
  #matplotlib.rcParams.update({'font.size': font_size, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})


  #fig1=plt.figure(figsize=(cm2inch(fig_width/2),cm2inch(fig_height)/1.2), dpi=400)

  marker_size = 50

  #plt.style.use(['science','no-latex'])

  fig, (ax1, ax2, ax3,ax4) = plt.subplots(1, 4)

  #fig.subplots_adjust(hspace=10)

  fig.set_size_inches(16,4)
  #ax1.set_aspect('equal')
  #fig.tight_layout(pad=6)
  fig.tight_layout(pad = 2)

  xlim = 1 #Number of Inverter [1/m]
  ylim = 2000 #Coil loss [W] 
  ax1.set_xlabel(r"$Number\ of\ inverter\ [1/m]$", labelpad = x_label_pad)              # not shown
  ax1.set_ylabel(r'$Coil loss [W]$', labelpad = y_label_pad) # not shown
  x1 = np.arange(0, xlim + 0.1,0.1)
  y1 = 0
  y2 = ylim
  plt_ax1 = ax1.scatter(accepted_n_inv_, accepted_coilloss, s=marker_size, c= color_list, rasterized=True)
  #fig.colorbar(mappable = plt_ax1 , ax = ax1, label=r"$B_{\rm stray}~[\rm \mu T]$")
  ax1.fill_between(x1, y1, y2 ,facecolor='r',alpha=0.3)
  ax1.plot([xlim, xlim], [0, ylim], 'r--', lw=0.5)  
  ax1.plot([0,xlim], [ylim, ylim], 'r--', lw=0.5) 
  ax1.axis([0,2, 0,5000])
  ax1.tick_params(axis='x', pad=15)
  # ax1.set_aspect('equal')



  xlim = 100
  ylim = 40
  ax2.set_xlabel(r"$B_{\rm stray}~[\rm \mu T]$", labelpad = x_label_pad)              # not shown
  ax2.set_ylabel(r'$Coupling [\%]$', labelpad = y_label_pad) # not shown

  x1 = np.arange(0, xlim + 0.1, 0.1)
  y1 = 0
  y2 = ylim

  plt_ax2 = ax2.scatter(accepted_bstray, accepted_kdiff,s=marker_size, c= color_list, rasterized=True)
  #fig.colorbar(mappable = plt_ax2 , ax = ax2, label=r"$V_{PriCore}$")
  ax2.fill_between(x1, y1, y2 ,facecolor='r',alpha=0.3)
  ax2.plot([xlim, xlim], [0, ylim], 'r--', lw=0.5)  
  ax2.plot([0,xlim], [ylim, ylim], 'r--', lw=0.5) 
  ax2.axis([0, 200, 0, 60])
  ax2.tick_params(axis='x', pad=15)

  # ax2.set_aspect('equal')

  xlim = 3500
  ylim = 30

  ax3.set_xlabel(r"$V_{SecCore}$", labelpad = x_label_pad)              # not shown
  ax3.set_ylabel(r'$P_{ave} [KW/M]$', labelpad = y_label_pad) # not shown

  x1 = np.arange(0, xlim + 0.1, 0.1)
  y1 = ylim
  y2 = ylim*10

  ax3.scatter(accepted_v_sec_core, accpeted_pave,s=marker_size, c= color_list, rasterized=True)
  #fig.colorbar(mappable = plt_ax3 , ax = ax3, label=r"$V_{SecWind}$")
  ax3.fill_between(x1, y1, y2 ,facecolor='r',alpha=0.3)
  ax3.plot([xlim, xlim], [ylim, ylim*10], 'r--', lw=0.5)  
  ax3.plot([0,xlim], [ylim, ylim], 'r--', lw=0.5) 
  ax3.axis([0, 5000, 0, 60])
  ax3.tick_params(axis='x', pad=15)
  # ax3.set_aspect('equal')


  xlim = 750
  ylim = 6000

  ax4.set_xlabel(r"$V_{SecWind}$", labelpad = x_label_pad)              # not shown
  ax4.set_ylabel(r'$V_{PriCore} $', labelpad = y_label_pad) # not shown

  x1 = np.arange(0, xlim + 0.1, 0.1)
  y1 = 0
  y2 = ylim

  ax4.scatter(accepted_v_sec_wind, accpeted_v_pri_core, s=marker_size, c= color_list, rasterized=True)
  #fig.colorbar(mappable = plt_ax3 , ax = ax3, label=r"$V_{SecWind}$")
  ax4.fill_between(x1, y1, y2 ,facecolor='r',alpha=0.3)
  ax4.plot([xlim, xlim], [0, ylim], 'r--', lw=0.5)  
  ax4.plot([0,xlim], [ylim, ylim], 'r--', lw=0.5) 
  ax4.axis([0, 2000, 0, 8000])
  ax4.tick_params(axis='x', pad=15)

  plt.suptitle("Epoch " +str(epoch).zfill(5) + "\nFound "+ str(len(accepted_bstray))+ " solutions out of 1000 input", y = 1.10)
  plt.savefig(FULL_PATH + "Solutions/"+str(epoch).zfill(5) + ".png")
  # "Epoch "+str(epoch).zfill(5) + "\nFound "+ str(len(accepted_bstray))+ " solutions out of 1000 input"
  plt.clf()

# Neural Network Training Loop

In [ ]:
kdiffs = []
bstrays = []
paves = []
coil_losses = []
vpricores = []
number_of_inverters_list = []
vsec_cores = []
accepted_solutions = []

i=0
combined_losses = []
cumulative_solutions = [0]
i = 1
number_of_epochs = 5001
max_solution = -float('inf')
for epoch in range(number_of_epochs):
    # Clear old gradients in the GP model
    for param in gp_model.parameters():
        param.grad = None

    # Create GP parameters from noise
    noise = generate_noise(shape=(1000, 10), gaussian = GAUSSIAN)
    gp = gp_model(noise)

    # Unpack and filter GP parameters
    a, lpx, lpy, ls, p, wp, ws, ip, np, ns, ys = scale_and_extract(gp)
    gp_parameters = stack_parameters(a, lpx, lpy, ls, p, wp, ws, *ys)
    extra_parameters = stack_parameters(ip, np, ns).transpose(0, 1)

    V_PriCore, V_SecCore, V_PriWind,V_SecWind = core_losses(extra_parameters,gp_parameters)
    new_number_of_inverters = number_of_inverters(gp_parameters)
    
    # Scale GP parameters - why?
    min_x = torch.min(gp_parameters)
    max_x = torch.max(gp_parameters)
    gp_parameters = (gp_parameters - min_x) / (max_x - min_x)
  
    # Push GP parameters through MP model
    mp_parameters = mp_model(gp_parameters)

    # Scale MP parameters
    y = torch.Tensor(df_mp.values).transpose(0, 1).to("cuda")
    min_y = torch.min(y, 1).values
    max_y = torch.max(y, 1).values
    mp_parameters = mp_parameters * (max_y - min_y) + min_y

    k_parameters = mp_parameters.transpose(0, 1)[0:6]
    l_parameters = mp_parameters.transpose(0, 1)[6:18]
    b_parameters = mp_parameters.transpose(0, 1)[18:]

    # Calculate losses
    kdiff = kdiff_loss(k_parameters)
    bstray = bstray_loss(k_parameters, l_parameters, b_parameters, extra_parameters)
    coilloss,pave = coil_loss_and_power_average(k_parameters, l_parameters, b_parameters, extra_parameters)

    kwargs = {}

    kwargs["kdiff"] = kdiff.clone().detach().cpu().numpy()
    kwargs["pave"] = pave.clone().detach().cpu().numpy()
    kwargs["coilloss"] = coilloss.clone().detach().cpu().numpy()
    kwargs["bstray"] = bstray.clone().detach().cpu().numpy()
    kwargs["vpricore"] = V_PriCore.clone().detach().cpu().numpy()
    kwargs["vsecwind"] = V_SecWind.clone().detach().cpu().numpy()
    kwargs["n_inv"] = new_number_of_inverters.clone().detach().cpu().numpy()
    kwargs["vseccore"] = V_SecCore.clone().detach().cpu().numpy()
    kwargs["epoch"] = epoch

    accepted_solution_so_far = accepted_solution(**kwargs)

    if epoch == 0:
      accepted_solution_so_far = 0
      
    elif epoch%100 == 0:
      accepted_solutions.append(accepted_solution_so_far)
      cumulative_solutions.append(cumulative_solutions[i - 1] + accepted_solution_so_far)
      i+=1
      generate_solution_plot(**kwargs)

    if max_solution < accepted_solution_so_far:
      torch.save(gp_model.state_dict(), f"./models/gp_model_{epoch:06}.pt")
      max_solution = accepted_solution_so_far

    if epoch % 1000 == 0:
      print("Accepted Soluntions : ", accepted_solution_so_far)
      plot_(**kwargs)
    # Scale losses
    #distance = distance_from_origin(kdiff, bstray, coilloss, V_SecCore, new_number_of_inverters) - distance_from_origin(pave)

    # distance = distance_from_origin(kdiff,bstray,coilloss) 
    # distance.mean().backward()

    new_kdiff = loss_function_relu(kdiff,.1502)
    new_bstray = loss_function_relu(bstray, 41.9)
    new_coil_loss = loss_function_relu(coilloss, 2000)
    new_pave = loss_function_power_avg(pave)
    new_number_of_inverters_ = loss_function_relu(new_number_of_inverters, 2)
    new_vsec_core = loss_function_relu(V_SecCore, 1500)
    new_v_pricore = loss_function_relu(V_PriCore, 6000)
    new_v_secwind = loss_function_relu(V_SecWind, 750)

    losses = [new_kdiff, new_bstray, new_coil_loss, new_pave, new_number_of_inverters_,new_vsec_core]

    total_loss = torch.sum(losses[pair[0]]*losses[pair[1]] + losses[pair[2]] * losses[pair[3]] + losses[pair[4]] *losses[pair[5]])

    # total_loss = torch.sum(new_kdiff + new_bstray + new_coil_loss + new_pave + new_number_of_inverters_
    #                        + new_vsec_core + new_pave)# + new_v_pricore + new_v_secwind)
    #total_loss = torch.sum(new_kdiff*new_bstray*new_v_pricore + new_coil_loss*new_number_of_inverters_*new_bstray + new_vsec_core*new_pave*new_v_secwind)

    # total_loss = torch.sum(new_bstray*new_coil_loss + 
    #                        + new_coil_loss*new_number_of_inverters_ 
    #                        + new_number_of_inverters_*new_bstray)

    # total_loss = torch.sum(new_kdiff*new_bstray + \
    #             new_coil_loss*new_number_of_inverters_ + new_number_of_inverters_*new_bstray + new_bstray*new_coil_loss + new_pave * new_vsec_core)
    
    # total_loss.backward()
    # optimizer.step()

    if epoch % 1000 == 0:
        # Store metrics for plotting or further logging
        kdiffs.append(torch.mean(kdiff).cpu().data.numpy())
        bstrays.append(torch.mean(bstray).cpu().data.numpy())
        #vpricores.append(torch.mean(V_PriCore).cpu().data.numpy())
        coil_losses.append(torch.mean(coilloss).cpu().data.numpy())
        paves.append(torch.mean(pave).cpu().data.numpy())
        number_of_inverters_list.append(torch.mean(new_number_of_inverters).cpu().data.numpy())
        
        print(f"Epoch: {epoch:4d}")
        #print(f"  Combined Loss:  {combined_losses[-1]:.10f}")
        print(f"  Kdiff :         {100 * kdiffs[-1]:.4f}%")
        print(f"  Bstray :        {bstrays[-1]:.4f}")
        print(f"  Pave :        {paves[-1]:.4f}")
        print(f"  Coilloss :        {coilloss[-1]:.4f}")
        print(f"  Inverters :        {number_of_inverters_list[-1]:.4f}")
        #print(f"  VPriCore :        {vpricores[-1]:.4f}")

        # Save model for further training or testing
        #torch.save(gp_model.state_dict(), f"./models/gp_model_{epoch:06}.pt")

In [ ]:
plt.plot([i*100 for i in range(len(cumulative_solutions))], cumulative_solutions)
plt.ylim(0,1000)
plt.xlim(0,5000)
plt.title("Cumulative Solution Plot", y = 1.10)
plt.xlabel("Epoch")
plt.ylabel("Total Solutions Found")
#plt.savefig("MeanLoss/" + "CumulativeSolution_MeanLoss"+".png")
plt.savefig("CumulativeSolution_MeanLoss.png")

In [ ]:
# mark = [i for i in range(0, number_of_epochs, 50)]
# marked_points = [accepted_solutions[mark[i]] for i in range(len(mark))]
accepted_solutions[0] = 0
max_value = max(accepted_solutions)
max_index = accepted_solutions.index(max_value)

plt.plot([ i*100 for i in range(len(accepted_solutions))], accepted_solutions)

# # plt.scatter(mark, marked_points, marker='o',color='y')
# plt.scatter([max_index], [max_value], marker = 'o', color = 'r')

# plt.annotate('Max number of solutions ('+ str(max_value)+') at epoch '+str(max_index), xy=(max_index,max_value), xycoords='data',
#             xytext=(-90,60), textcoords='offset points',
#             arrowprops=dict(arrowstyle='fancy',fc='0.2',
#                             connectionstyle="angle3,angleA=45,angleB=90"))
plt.xlabel("Number of epochs")
plt.ylabel("Accepted solutions")
plt.savefig(FULL_PATH + "soultions_per_epoch.png")
plt.clf()


In [ ]:
# from os import walk

# filenames = next(walk("images/"), (None, None, []))[2]
# filenames.sort(key=lambda f: int(f.split(".")[0]))
# with imageio.get_writer('images/mygif.gif', mode='I') as writer:
#   for filename in filenames:
#       image = imageio.imread("images/"+filename)
#       writer.append_data(image)

In [ ]:
with torch.no_grad():
  noise = generate_noise(shape=(1000, 10), gaussian = GAUSSIAN)
  # gp_model = GpModel(10).to("cuda")
  # gp_model.load_state_dict(torch.load("gp_model_000204.pt"))
  gp = gp_model(noise)
  # Unpack and filter GP parameters
  a, lpx, lpy, ls, p, wp, ws, ip, np, ns, ys = scale_and_extract(gp)
  gp_parameters = stack_parameters(a, lpx, lpy, ls, p, wp, ws, *ys)
  g = gp_parameters.clone()
  #print(gp_parameters)
  extra_parameters = stack_parameters(ip, np, ns).transpose(0, 1)
  V_PriCore, V_SecCore, V_PriWind,V_SecWind = core_losses(extra_parameters,gp_parameters)
  new_number_of_inverters = number_of_inverters(gp_parameters)
  # Scale GP parameters - why?
  min_x = torch.min(gp_parameters)
  max_x = torch.max(gp_parameters)
  gp_parameters = (gp_parameters - min_x) / (max_x - min_x)

  # Push GP parameters through MP model
  mp_parameters = mp_model(gp_parameters)

  # Scale MP parameters
  y = torch.Tensor(df_mp.values).transpose(0, 1).to("cuda")
  min_y = torch.min(y, 1).values
  max_y = torch.max(y, 1).values
  mp_parameters = mp_parameters * (max_y - min_y) + min_y

  k_parameters = mp_parameters.transpose(0, 1)[0:6]
  l_parameters = mp_parameters.transpose(0, 1)[6:18]
  b_parameters = mp_parameters.transpose(0, 1)[18:]

  # Calculate losses
  kdiff = kdiff_loss(k_parameters)
  bstray = bstray_loss(k_parameters, l_parameters, b_parameters, extra_parameters)
  coilloss,pave = coil_loss_and_power_average(k_parameters, l_parameters, b_parameters, extra_parameters)
  kwargs = {}

  kwargs["kdiff"] = kdiff.clone().detach().cpu().numpy()
  kwargs["pave"] = pave.clone().detach().cpu().numpy()
  kwargs["coilloss"] = coilloss.clone().detach().cpu().numpy()
  kwargs["bstray"] = bstray.clone().detach().cpu().numpy()
  kwargs["vpricore"] = V_PriCore.clone().detach().cpu().numpy()
  kwargs["vsecwind"] = V_SecWind.clone().detach().cpu().numpy()
  kwargs["n_inv"] = new_number_of_inverters.clone().detach().cpu().numpy()
  kwargs["vseccore"] = V_SecCore.clone().detach().cpu().numpy()
  kwargs["epoch"] = 1

  #plot_(**kwargs)

  print(accepted_solution(**kwargs))

In [ ]:
min(kwargs["vsecwind"])

In [ ]:
generate_solution_plot(**kwargs)

In [ ]:
# from prettytable import PrettyTable
# t = PrettyTable(['Number of Inverters [1/m]', 'Coilloss [W]', 'Bstray[uT]', 'Coupling [%]', 'V_Sec_Core [cm3]', 'Power Average [kW/m'])
# for i in range(len(accepted_n_inv_)):
#   t.add_row([accepted_n_inv_[i], accepted_coilloss[i],accepted_bstray[i],accepted_kdiff[i],accepted_v_sec_core[i], accpeted_pave[i]])
# print(t)